In [19]:
import torch
import torchvision
import executorch
import executorch.exir

In [3]:
# Load pretrained ViT
num_labels = 11  # Get number of labels (e.g., 8)

model = torchvision.models.vit_b_16(weights="IMAGENET1K_V1")  # Load a pretrained model
model.heads.head = torch.nn.Linear(model.heads.head.in_features, num_labels)  # Modify last layer

In [5]:
# Load state dict
model.load_state_dict(torch.load("models/tomato_model_2025_02_28_v2.pt", weights_only=True))

<All keys matched successfully>

In [23]:
# 1. torch.export: Defines the program with the ATen operator set
dummy_image = torch.ones((1, 3, 224, 224))
#dummy_image.size()
torch.nn.functional.softmax(model(dummy_image), dim=1)

aten_dialect = torch.export.export(model, args=(dummy_image,))


In [24]:
# 2. to_edge: Make optimizations for Edge devices
edge_program = executorch.exir.to_edge(aten_dialect)
edge_program


In [25]:
# 3. to_executorch: Convert the graph to an ExecuTorch program
executorch_program = edge_program.to_executorch()
executorch_program


In [26]:
# 4. Save the compiled .pte program
with open("models/tomato_model_2025_02_28_v2_executorch.pte", "wb") as file:
    file.write(executorch_program.buffer)